In [1]:

from sentence_transformers import SentenceTransformer
import pandas as pd
import time
import numpy as np

# Load embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\AbhinavKasubojula\OneDrive - Kenall Inc\Desktop\code\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\AbhinavKasubojula\OneDrive - Kenall Inc\Desktop\code\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
import os
import faiss
import numpy as np
import pickle

# Load the FAISS index
storage_directory = r'C:\Users\AbhinavKasubojula\OneDrive - Kenall Inc\Desktop\code\stored_data'
index = faiss.read_index(os.path.join(storage_directory, 'faiss_index.bin'))

# Load document texts
with open(os.path.join(storage_directory, 'documents.pkl'), 'rb') as f:
    documents = pickle.load(f)

# Load metadata
with open(os.path.join(storage_directory, 'metadata.pkl'), 'rb') as f:
    file_metadata = pickle.load(f)

# Ensure documents and metadata are loaded correctly
print(f"Loaded {len(documents)} documents.")
print(f"Loaded {len(file_metadata)} metadata entries.")

# Function to retrieve text and documents based on a query
def retrieve_text_and_doc(query, k=1):
    # Step 1: Compute the query embedding
    query_embedding = embedding_model.encode([query], convert_to_tensor=False)
    query_embedding = query_embedding / np.linalg.norm(query_embedding)

    # Check embedding size and FAISS index dimensionality
    assert query_embedding.shape[1] == index.d, f"Embedding size {query_embedding.shape[1]} does not match FAISS index dimensionality {index.d}"

    # Step 2: Perform the search in the FAISS index
    distances, indices = index.search(np.array(query_embedding).astype('float32'), k)

    # Step 3: Retrieve the corresponding documents and metadata
    retrieved_docs = []
    retrieved_metadata = []
    
    for i in indices[0]:
        if i < len(documents):
            retrieved_docs.append(documents[i])
            retrieved_metadata.append(file_metadata[i])
        else:
            print(f"Warning: Index {i} is out of range for documents. Total documents: {len(documents)}.")

    return retrieved_docs, retrieved_metadata, distances[0]

# Example usage
query = input("Enter your query: ")
retrieved_docs, retrieved_metadata, distances = retrieve_text_and_doc(query)

# Join and display retrieved text and metadata
retrieved_text = " ".join(retrieved_docs)
print(query)
for doc, meta, dist in zip(retrieved_docs, retrieved_metadata, distances):
    print(f"Metadata: {meta}, Distance: {dist}, Document: {doc[:100]}...")


Loaded 6 documents.
Loaded 6 metadata entries.
W912EP22R0042
Metadata: {'doc_number': 5, 'doc_name': 'W912BV24R0001_Synopsis_99M_General_Services_IDC.pdf'}, Distance: 1.6505262851715088, Document: INDEFINITE DELIVERY CONTRACT (IDC) A-E SERVICES FOR USE WITHIN
SOUTHWESTERN DIVISION (SWD), PROJECTS...


In [8]:
print(file_metadata)

[{'doc_number': 1, 'doc_name': 'KC USACE_B_01_03_2024_AFCS_AE_Synopsis_SB_Set_Aside_W912DQ24R4019 (1) (1).pdf'}, {'doc_number': 2, 'doc_name': 'Revised+Synposis++-+6.14.23 (1).pdf'}, {'doc_number': 3, 'doc_name': 'Solicitation Vicksburg.pdf'}, {'doc_number': 4, 'doc_name': 'USACE Jacksonville SB AE.pdf'}, {'doc_number': 5, 'doc_name': 'W912BV24R0001_Synopsis_99M_General_Services_IDC.pdf'}, {'doc_number': 6, 'doc_name': 'W912DQ24R4002_2024_Military_SB_AE_Synopsis.pdf'}]


In [9]:
retrived_text = " ".join(retrieved_docs)
print(retrived_text)

INDEFINITE DELIVERY CONTRACT (IDC) A-E SERVICES FOR USE WITHIN
SOUTHWESTERN DIVISION (SWD), PROJECTS ASSIGNED TO SWD, AND
PRIMARILY IN SUPPORT OF TULSA DISTRICT (SWT) MILITARY, CIVIL WORKS,
AND INTERNATIONAL AND INTERAGENCY SERVICES (IIS)
Synopsis W912BV24R0001
1. CONTRACT INFORMATION:
General: This A-E Services Contract is being procured In Accordance With (IAW) 40
USC Chapter 11 Selection of Architects and Engineers (formerly known as the Brooks
Act) and Federal Acquisition Regulation (FAR) Subpart 36.6. Firms will be selected for
negotiation based on demonstrated competence and qualifications for the required
work. This announcement is being advertised as “Unrestricted” under NAICS code
541330, engineering services that has a small business size standard of $25,500,000.
This announcement is for a Multiple Award Task Order Contract (MATOC) for A-E
services with a total shared capacity of $99,000,000 and includes a target of five (5)
Unrestricted A-E Indefinite Delivery Contracts (IDC

In [11]:
import requests
import json

# Initialize an empty list to store previous questions and answers
memory = []

# Pinecone text retrieval part (you've done this)
def generate_response(cx, q):
    global memory

    # Retrieved text as context from Pinecone
    retrieved_text = cx

    # Prepare the prompt with memory of past interactions
    memory_prompt = "\n".join(memory)  # Join the previous interactions
    question = q
    prompt = f"{memory_prompt}\n look at the previous answer you gave for the question {query} the last answer you gave is eligibilty criteria to get tender picked up. now answer the qustion find professional qualification from this context\n{retrieved_text}\n\n"

    #prompt = f"Here is some context:\n{retrieved_text}\n{memory_prompt}\n\nNow, based on this, can you answer the following question: {question}"

    # Ollama API call to generate a response
    url = "http://localhost:11434/api/generate"
    headers = {"Content-Type": "application/json"}
    data = {
        "model": "llama3.1:latest",
        "prompt": prompt,
        "stream": False  # Disable streaming
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))

    # Check the response from the model
    if response.status_code == 200:
        result = response.json()
        ans = result['response']
        print("Model response:", ans)

        # Store the current question and answer in memory
        memory.append(f"Q: {question}\nA: {ans}")
        if len(memory) > 5:  
            memory = memory[-5:]

        return ans
    else:
        print("Error:", response.text)
        return None




In [12]:
answer = generate_response(retrieved_text, "give me contract information ")

Model response: It appears you've provided a detailed set of instructions and requirements for submitting proposals for an A-E contract, specifically in response to a solicitation announcement.

To ensure I understand correctly, here's a summary of what I've extracted from these instructions:

**Project Requirements:**

1. A maximum of 10 projects can be submitted.
2. Each project must have been performed within the last 10 years.
3. Projects with recent designs (at least 75% completed) prior to the submission due date are eligible.
4. Projects are defined as single Task Orders against an IDC ("D" contract), or "C" type contracts covering work at one site, multiple sites within a single installation/facility, or multiple but separate locations.

**Section F Requirements:**

1. The Section F of the SF 330 shall not exceed 20 pages.
2. Each submitted project must meet one of the project definitions listed above.

**Other Requirements:**

1. In Section G, Block 26, include the name, firm 

In [7]:
print(retrieved_text)

An o icial website of the United States government Here’s how you know
 Follow
Contract Opportunity
General Information
Classification
Description
Attachments/Links
Contact Information
History
Award Notices
W912EP22R0042 INDEFINITE DELIVERY (MATOC) FOR MULTI- DISCIPLINE MISCELLANEOUS MILITARY WORKS DESIGN AND OTHER ARCHITECT AND ENGINEERING (A-E) SERVICES WITHIN THE FLORIDA BOUNDARIES OF THE JACKSONVILLE DISTRICT, PUERTO RICO AND THE U.S. VIRGIN ISLANDS
ACTIVE
Contract Opportunity
Notice ID W912EP22R0042
Related Notice
Department/Ind. Agency DEPT OF DEFENSE Sub-tier DEPT OF THE ARMY
DEPT OF THE ARMY Major Command USACE Sub Command SAD Oﬀice US ARMY ENGINEER DISTRICT JACKSONVI
General Information
View Changes
Contract Opportunity Type: Solicitation (Updated)
All Dates/Times are: (UTC-04:00) EASTERN STANDARD TIME, NEW YORK, USA Updated Published Date: Aug 23, 2022 10:38 am EDT
Original Published Date: Aug 23, 2022 10:17 am EDT
Updated Date Oﬀers Due: Sep 23, 2022 12:00 pm EDT
Original 

In [8]:
# Example usage:
context = retrived_text
question = query
answer = generate_response(context, question)

In [4]:
answer = generate_response("did you remember your last 3 responses?", "?")

In [ ]:
print(query)